In [ ]:
# Preprocessing: caricamento, pulizia, feature engineering e encoding

# Imports

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder

# Parte principale
Questa funzione esegue il caricamento e il preprocessing dei dati per un problema di classificazione multiclasse. Restituisce i dataset pronti per essere usati in fase di modellazione (training e test).

In [ ]:

def load_and_preprocess():
    # Caricamento dei file CSV
    '''
    - Carica i dati di training (train_values.csv) e le etichette (train_labels.csv) usando building_id come indice.

    - Carica anche il dataset di test (test_values.csv).

    - Estrae la colonna target label oppure la prima colonna se il nome non è specificato.
    '''
    BASE = Path(__file__).parent
    X = pd.read_csv(BASE / 'train_values.csv', index_col=0)
    y_df = pd.read_csv(BASE / 'train_labels.csv', index_col=0)
    y = y_df['label'] if 'label' in y_df.columns else y_df.iloc[:, 0]
    X_test = pd.read_csv(BASE / 'test_values.csv', index_col=0)

    # Conversione oggetti in categorie
    '''
    - Identifica le colonne di tipo object (stringhe) e le converte in category, sia per X che per X_test.
    '''
    cat_cols = X.select_dtypes(include='object').columns.tolist()
    for c in cat_cols:
        X[c] = X[c].astype('category')
        if c in X_test.columns:
            X_test[c] = X_test[c].astype('category')

    # Rimozione colonne costanti
    '''
    - Elimina le colonne che hanno un solo valore unico (costanti), perché non contribuiscono all'apprendimento del modello.
    '''
    nunique = X.nunique()
    low_var = nunique[nunique <= 1].index.tolist()
    X.drop(columns=low_var, inplace=True)
    X_test.drop(columns=low_var, inplace=True, errors='ignore')
    cat_cols = [c for c in cat_cols if c not in low_var]

    # Imputazione e clipping outlier
    '''
    - Identifica le colonne numeriche (int64 o float64) e applica imputazione con la mediana.

    - Applica un clipping sui valori numerici per ridurre l'influenza degli outlier, usando il 0.15 percentile inferiore e il 99.87 superiore.
    '''
    num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    imp = SimpleImputer(strategy='median')
    X[num_cols] = imp.fit_transform(X[num_cols])
    X_test[num_cols] = imp.transform(X_test[num_cols])

    for col in num_cols:
        lo = X[col].quantile(0.00153)
        hi = X[col].quantile(0.9987)
        X[col] = X[col].clip(lo, hi)
        X_test[col] = X_test[col].clip(lo, hi)

    # Feature ingegnerizzate geografiche
    '''
    - Crea nuove feature combinate tra i livelli geografici: somme, prodotti e divisioni pairwise, più un prodotto totale (geo_prod_123).

    - Evita la divisione per zero usando eps = 1e-5.
    '''
    eps = 1e-5
    for a, b in [(1,2), (1,3), (2,3)]:
        X[f'geo_sum_{a}{b}'] = X[f'geo_level_{a}_id'] + X[f'geo_level_{b}_id']
        X[f'geo_prod_{a}{b}'] = X[f'geo_level_{a}_id'] * X[f'geo_level_{b}_id']
        X_test[f'geo_sum_{a}{b}'] = X_test[f'geo_level_{a}_id'] + X_test[f'geo_level_{b}_id']
        X_test[f'geo_prod_{a}{b}'] = X_test[f'geo_level_{a}_id'] * X_test[f'geo_level_{b}_id']

    X['geo_prod_123'] = X['geo_level_1_id'] * X['geo_level_2_id'] * X['geo_level_3_id']
    X_test['geo_prod_123'] = X_test['geo_level_1_id'] * X_test['geo_level_2_id'] * X_test['geo_level_3_id']

    for a, b in [(1,2), (1,3), (2,3)]:
        X[f'geo_div_{a}{b}'] = X[f'geo_level_{a}_id'] / (X[f'geo_level_{b}_id'] + eps)
        X_test[f'geo_div_{a}{b}'] = X_test[f'geo_level_{a}_id'] / (X_test[f'geo_level_{b}_id'] + eps)

    # Encoding
    '''
    Applica un ColumnTransformer con due trasformazioni:

    - passthrough per le colonne numeriche

    - OrdinalEncoder per le colonne categoriche, gestendo eventuali categorie sconosciute nei dati di test.
    '''
    num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    preprocessor = ColumnTransformer([
        ('num', 'passthrough', num_cols),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_cols)
    ])
    X_enc = preprocessor.fit_transform(pd.concat([X[num_cols], X[cat_cols]], axis=1))
    X_test_enc = preprocessor.transform(pd.concat([X_test[num_cols], X_test[cat_cols]], axis=1))
    # Output della funzione
    '''
    Restituisce:

    - X_enc: dati di training preprocessati e codificati

    - y: etichette corrispondenti

    - X_test_enc: dati di test preprocessati e codificati

    - X_test.index: indice dei dati di test, utile per ricostruire il file di submission
    '''
    return X_enc, y, X_test_enc, X_test.index
